In [1]:
import os
import time
import cv2
import requests
import json
import numpy as np
import tensorflow as tf
from models.config import TABLE_PATH, TEST_PATH, OUTPUT_SHAPE

## 配置信息

In [2]:
#字典
with open(TABLE_PATH, 'r',encoding='utf8') as f:
    inv_table = [char.strip() for char in f]+[' ']*2

In [3]:
#测试数据集
test_all_image_paths = [TEST_PATH + img for img in sorted(os.listdir(TEST_PATH))]

## 调用tensorflow serving方法

In [4]:
def text_ocr(image_list):
    '''
    矩形框文字识别
    '''
    result_list = []
    input_image = []
    for img in image_list:
        image = cv2.resize(img, (OUTPUT_SHAPE[1], OUTPUT_SHAPE[0]), 3)
        image = image.reshape(OUTPUT_SHAPE)
        input_image.append(image.tolist())
    url = 'http://192.168.46.230:8501/v1/models/crnn:predict'
    headers = {"content-type": "application/json"}
    data = json.dumps({
            "signature_name": 'serving_default',
            "instances":input_image})
    predictions = requests.post(url,data=data, headers=headers).json()
    if 'predictions' in predictions:
        if len(predictions['predictions']) > 0:
            for sentense in predictions['predictions']:
                result = ''
                for i in range(len(predictions['predictions'][0])):
                    char_index = np.argmax(predictions['predictions'][0][i])
                    if char_index != len(inv_table):
                        result +=  inv_table[char_index]
                result_list.append(result)
    return result_list

## 准备图片

In [ ]:
image = tf.io.read_file(test_all_image_paths[4])
image = tf.image.decode_jpeg(image, channels=3)
image = tf.image.resize(image, [32, 280]).numpy()
image_list = [image]*100

## 调用接口

In [ ]:
start = time.time()
result = text_ocr(image_list)
print('识别时间：', time.time()-start)
print('识别结果：', result)